<a href="https://colab.research.google.com/github/Sakinat-Folorunso/OOU_CSC309_Artificial_Intelligence/blob/main/notebooks/CSC309_W01_Turing_Test_Practical_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 1 Lab:** Turing Test Practical — Build a Rule‑based Chatbot

**Instructor:** Dr Sakinat Folorunso

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> In this lab you will create and extend a tiny ELIZA‑style chatbot, then run a mini **Turing Test**.
> Every code cell is commented line‑by‑line so you understand exactly what it does.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.
2. Run the **Setup** cell once.
3. Work through the tasks. Edit only cells labeled **`# TODO(Student)`**.
4. Use the **Quick Checks** to test your code.
5. Finish with the **Reflection**. If you are done early, try the **Extensions**.

**Assessment focus:** clarity of rules, correctness of memory feature, and quality of your transcript.

In [ ]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# This cell collects your group information using Colab form fields.
# The '#@title' line makes a nice header above the cell UI.
# The '#@param' annotations create text boxes in Colab for easy input.

group_members = "Type names here"  #@param {type:"string"}  # Names of all group members (comma‑separated)
scenario = "One sentence describing your chatbot use‑case (e.g., student help desk)"  #@param {type:"string"}  # Your real‑world scenario

# The print statements give immediate confirmation of what you entered.
print("👥 Group members:", group_members)  # Show the group roster
print("🧩 Scenario:", scenario)            # Show the chosen scenario

## Do Now (2–3 mins)
Discuss in pairs: *When might a rule‑based chatbot be convincing? When might it fail?*  
Add one sentence to your **Group Log** that names your use‑case (e.g., timetable Q&A).

In [ ]:
#@title 🔧 Setup (run once per session)
# This cell ensures any needed packages are available in Colab.
# For Week 1 we only use Python's standard library modules (no extra installs).

import sys  # Gives access to Python interpreter details (not strictly needed here)
import re   # Regular expressions for pattern matching (core of our rule system)
import random  # To randomly pick between multiple plausible responses

# We show a simple confirmation message so you know setup ran successfully.
print("✅ Setup complete. You're ready to build a chatbot!")

## Mini‑notes: How this chatbot works
A rule‑based chatbot uses **pattern matching** to map input text to template responses.  
We’ll write a small set of regular‑expression rules and a **fallback** for anything unmatched.

In [ ]:
#@title 🗣️ Starter chatbot (ELIZA‑style)
# The code below implements a tiny rule-based chatbot.
# Every line is commented to explain what it does.

# We create a list of (pattern, handler) pairs. Each pattern is a compiled regular
# expression. Each handler is a function that takes a regex match object and returns a string.
rules = [
    # 1) Greetings rule: if the user says 'hello', 'hi', or 'hey' (case-insensitive)
    #    we respond with one of two friendly prompts.
    (re.compile(r'\b(hello|hi|hey)\b', re.I),  # re.I makes the match case-insensitive
     lambda m: random.choice([                 # random.choice picks one reply at random
         "Hello! How can I help today?",
         "Hi there—what's on your mind?"
     ])),

    # 2) Emotion rule: capture whatever follows 'I feel ...' and reflect it back.
    #    The '(.*)' part creates a group that matches the rest of the line.
    (re.compile(r'i feel (.*)', re.I),
     lambda m: f"Why do you feel {m.group(1)}?"),

    # 3) Question rule: if the input ends with a '?', give a reflective prompt.
    (re.compile(r'(.*)\?$', re.I),
     lambda m: "That's a good question. What do you think?"),
]

def eliza_reply(text: str) -> str:
    """Return a reply for the user's input using rule-based matching."""
    text = text.strip()                 # Remove leading/trailing whitespace to normalize input
    for pattern, handler in rules:      # Loop over all (pattern, handler) pairs
        match = pattern.search(text)    # Try to find the pattern anywhere in the text
        if match:                       # If we found a match
            return handler(match)       # Call the handler to generate a response
    # If no rule matched, return a generic prompt encouraging the user to say more.
    return random.choice([
        "Tell me more…",
        "How does that relate to your studies?",
        "What would make this better?"
    ])

# Quick demonstration of the starter rules:
examples = ["hello", "I feel nervous about exams", "Is AI dangerous?"]  # Sample user inputs
for s in examples:                            # Iterate through the sample inputs
    print(f"> {s}")                           # Print the user's message
    print(eliza_reply(s))                     # Print the chatbot's response
    print()                                   # Blank line for readability

## Task 1 — Extend the chatbot with **memory** and two new rules
1. If the user says **“my name is …”**, store the name and greet them personally next time.  
2. Add **two** more patterns that would be useful for your scenario (e.g., thanks/bye, course code questions).  
3. Keep your explanations in the comments so future you (and your teammates) understand the code.

In [ ]:
# TODO(Student): Add memory + at least two new rules
# We implement a new function 'eliza_reply_plus' that wraps the original 'eliza_reply'
# and adds: (a) name memory and (b) two extra pattern handlers.

# A global variable (for simplicity) holds the user's name if provided.
user_name = None  # Initially, we don't know the user's name

def eliza_reply_plus(text: str) -> str:
    """Extended version with name memory and extra rules."""
    global user_name                      # We will update the global 'user_name' variable
    text = text.strip()                   # Normalize input by trimming whitespace

    # --- New Rule A: Capture and remember the user's name --------------------
    # Pattern: 'my name is <word>'. We capture the name in group(1).
    m = re.search(r'\bmy name is ([A-Za-z]+)\b', text, re.I)  # Look for the name phrase (letters only for simplicity)
    if m:                                                       # If the pattern matched
        user_name = m.group(1).title()                          # Store a title-cased version of the name
        return f"Nice to meet you, {user_name}. How can I help?"  # Personal greeting

    # If we already know the user's name and they greet us, greet them back personally.
    if user_name and re.search(r'\b(hello|hi|hey)\b', text, re.I):  # Detect a greeting with known name
        return f"Hi {user_name}! What's next?"                       # Personalized response

    # --- New Rule B: Gratitude detection -------------------------------------
    # Many conversations include 'thank you' or 'thanks' — acknowledge politely.
    if re.search(r'\b(thank you|thanks)\b', text, re.I):             # Detect gratitude words
        return "You're welcome! Is there anything else I can help with?"  # Friendly acknowledgement

    # --- New Rule C: Goodbye detection ---------------------------------------
    # End the conversation gracefully when the user says 'bye' or 'goodbye'.
    if re.search(r'\b(bye|goodbye)\b', text, re.I):                  # Detect common goodbye variations
        return "Goodbye! Have a great day."                           # Courteous closing

    # If none of the new rules apply, fall back to the original rule set.
    return eliza_reply(text)                                           # Delegate to base rules

### Quick Check — Guided tests
Run the cell below to make sure your memory rule works.

In [ ]:
# The list 'dialogue' simulates a short conversation to test memory and new rules.
dialogue = [
    "my name is Ada",           # Introduce a name (should be stored)
    "hi",                       # Greeting should now be personalized
    "thank you",                # Gratitude should trigger a polite reply
    "bye"                       # Farewell should trigger a closing
]

# We iterate through each message and print the bot's response.
for line in dialogue:                              # For every simulated user message
    print(f"User: {line}")                         # Show the user input
    reply = eliza_reply_plus(line)                 # Compute the chatbot's reply using the extended function
    print(f"Bot : {reply}")                        # Show the bot output
    print()                                        # Blank line for readability

## Task 2 — **Turing Test** Activity (10–15 min)
1. One student chats with the bot for ~1 minute while others watch the transcript.  
2. Observers decide if it *could* be human and list **what made it convincing** and **what gave it away**.  
3. Rotate roles and try to improve your rules based on feedback.

In [ ]:
# TODO(Student): Produce a short transcript that demonstrates your rules and memory.
# You can edit the 'user_inputs' list below to craft a representative dialogue.

user_inputs = [
    "hello",
    "my name is Ada",
    "hi",
    "I feel overwhelmed by assignments",
    "Is AI the same as machine learning?",
    "thanks",
    "goodbye"
]

# We run the conversation and print a neat transcript showing both sides.
for msg in user_inputs:                       # Loop over each user message
    print(f"User: {msg}")                     # Show user input
    print(f"Bot : {eliza_reply_plus(msg)}")   # Show bot output using our extended reply function
    print()                                   # Add a blank line for readability

## Reflection (5–7 sentences)
- When did the bot seem **convincing**? When did it obviously fail?  
- Which two rules mattered most for your scenario?  
- What would you change next time (e.g., add context, handle negation, track topics)?

In [ ]:
#@title 📝 Reflection (edit the text inside the triple quotes)
reflection = """
Write your 5–7 sentence reflection here.
- What felt realistic? What broke the illusion?
- How did your 'memory of name' change the tone of the conversation?
- Which additional rule would deliver the biggest improvement for your chosen scenario?
"""

# Printing helps ensure your reflection is saved in the notebook output.
print(reflection.strip())  # Show the reflection text as entered by the student

## Extensions (if you finish early)
- Add a **topic tracker** (e.g., if the user mentions 'exam', ask a follow‑up).  
- Implement a tiny **FAQ table** (dictionary) for your scenario (e.g., office hours).  
- Log the last **3 messages** to create context for follow‑up questions.